# Preparing to play

This notebook has my work towards an implementation of the play function in play.py

Initial imports

In [99]:
import pandas as pd
import numpy as np
from processing import *

Get Jeopardy data and preprocess it

In [100]:
jeopardy_data = pd.read_csv("jeopardy.csv")
preprocess(jeopardy_data)

Ensure each category has 5 or more questions, and pick 7 random categories out of those available.

In [101]:
# Makes it easier to change the variables later
categories_to_show = 7
min_questions = 5

In [102]:
categories = jeopardy_data.category
categories = categories.value_counts().apply( \
    lambda c: c >= min_questions)
categories = categories[categories == True]
categories = categories.sample(categories_to_show)
categories = categories.index
categories

Index(['CHILDREN'S BOOK TITLES', 'PROTESTANTISM', 'MOTHER'S DAY',
       'THAT'S THE SPOT', 'THE MAP OF SOUTH AMERICA',
       'BROADWAY CROSSWORD CLUES "T"', 'WEAPONS OF THE 2nd MILLENNIUM'],
      dtype='object')

Get 5 questions for the available categories

In [103]:
questions = jeopardy_data[jeopardy_data.category.apply( \
    lambda c: c in categories)]
category_question_count = {}
def incr(key):
    category_question_count.setdefault(key, 0)
    category_question_count[key] += 1
    return category_question_count[key] <= 5
questions = questions[questions.category.apply(incr)].reset_index(drop=True)
questions

,show_number,air_date,round,category,value,question,answer,value_float,air_year
0,5460,2008-05-09,Jeopardy!,MOTHER'S DAY,$200,"This type of ""mom"" drives her kids around in a...",a soccer mom,200.0,2008
1,5460,2008-05-09,Jeopardy!,MOTHER'S DAY,$400,"Mom's traditional warning, about getting these...",cramps,400.0,2008
2,5460,2008-05-09,Jeopardy!,MOTHER'S DAY,$600,Dictionaries don't have a word for the study o...,Jesus,600.0,2008
3,5460,2008-05-09,Jeopardy!,MOTHER'S DAY,$800,A mom who pushes her child into the performing...,a stage mother,800.0,2008
4,5460,2008-05-09,Jeopardy!,MOTHER'S DAY,$1000,"This 19th century woman was mom to Clara, Will...",Mary (Wollstonecraft) Shelley,1000.0,2008
5,4612,2004-09-28,Jeopardy!,"BROADWAY CROSSWORD CLUES ""T""",$200,It's how modern Millie is (10),thoroughly,200.0,2004
6,4612,2004-09-28,Jeopardy!,"BROADWAY CROSSWORD CLUES ""T""",$400,River City situation (7),trouble,400.0,2004
7,4612,2004-09-28,Jeopardy!,"BROADWAY CROSSWORD CLUES ""T""",$600,"""No, No, Nanette"" ""beverage"" song (3, 3, 3)","""Tea For Two""",600.0,2004
8,4612,2004-09-28,Jeopardy!,"BROADWAY CROSSWORD CLUES ""T""",$800,"""Aida"" lyricist's first name (3)",Tim,800.0,2004
9,4612,2004-09-28,Jeopardy!,"BROADWAY CROSSWORD CLUES ""T""",$1000,"""My One And Only"" tall Tony-winning Texan (5, 4)",Tommy Tune,1000.0,2004


Print out the categories and money values for each question in the category

In [104]:
# Terminals are typically 80 characters wide
view_width = 80
cell_width, expanded_cells = divmod(view_width, categories_to_show)
cell_width -= 1  # Add space for a pipe character
cell_template_str = "{{:^{}}}|"
cell_format_str = cell_template_str.format(cell_width)

In [105]:
longest_category = max(categories, key=len)
lines, extra_line = divmod(len(longest_category), cell_width)
if extra_line > 0:
    lines += 1

for line in range(lines):
    extender = expanded_cells
    for col in categories:
        start = line * cell_width
        end = start + cell_width
        print(cell_format_str.format(col[start:end]), sep="", end="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print()
print("-" * view_width)  # Separator between categories and monies

CHILDREN'S| PROTESTANT| MOTHER'S D| THAT'S THE|THE MAP OF|BROADWAY C|WEAPONS OF|
 BOOK TITL|    ISM    |     AY    |    SPOT   | SOUTH AME|ROSSWORD C| THE 2nd M|
    ES    |           |           |           |   RICA   | LUES "T" |ILLENNIUM |
--------------------------------------------------------------------------------


Print out the money values for each selected question

In [106]:
for row in range(min_questions):
    extender = expanded_cells
    for col in categories:
        money_number = questions.loc[questions.category == col].value_float.iloc[row]
        if money_number is None:
            money_number = row * 400
        money_value = questions.loc[questions.category == col].value.iloc[row]
        if money_value is None:
            money_value = "${}".format(money_number)
        print(cell_format_str.format(money_value), end="", sep="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print("\n", "-" * view_width, sep="")

   $400   |    $200   |    $200   |    $200   |   $200   |   $200   |   $400   |
--------------------------------------------------------------------------------
   $800   |    $400   |    $400   |    $400   |   $400   |   $400   |   $800   |
--------------------------------------------------------------------------------
  $1200   |    $600   |    $600   |    $600   |   $600   |   $600   |  $1200   |
--------------------------------------------------------------------------------
  $1600   |    $800   |    $800   |    $800   |  $1,000  |   $800   |  $7,800  |
--------------------------------------------------------------------------------
  $2000   |   $1000   |   $1000   |   $1000   |  $1000   |  $1000   |  $2000   |
--------------------------------------------------------------------------------


Prompt for category and money value

In [115]:
wallet = 0
answers_given = 0
max_answers = categories_to_show * min_questions
def play_one_question():
    global wallet  # nonlocal wallet
    if answers_given > max_answers:
        return False
    print("Your wallet: ${}".format(wallet))

    # Prompt for category
    print("Select category (1-{}, "
          "from left to right, or Q to quit)" \
          .format(categories_to_show),
          sep="", end="\n>>> ")
    category_index_choice = None
    while type(category_index_choice) != int:
        category_index_choice = input()
        if category_index_choice.lower() == "q":
            return False
        try:
            category_index_choice = int(category_index_choice)
        except ValueError:
            print("Invalid choice. Please enter a number.")
            continue
        if (category_index_choice > categories_to_show or
                category_index_choice < 1):
            print("Invalid choice.")
            category_index_choice = None
    category_index_choice -= 1  # 0-based indices

    # Prompt for question
    print("Select question (1-{}, "
          "from top to bottom, or Q to quit)" \
          .format(min_questions),
          sep="", end="\n>>> ")
    question_index_choice = None
    while type(question_index_choice) != int:
        question_index_choice = input()
        if question_index_choice.lower() == "q":
            return False
        try:
            question_index_choice = int(question_index_choice)
        except ValueError:
            print("Invalid choice. Please enter a number.")
            continue
        if (question_index_choice > min_questions or
                question_index_choice < 1):
            print("Invalid choice.")
            question_index_choice = None
    question_index_choice -= 1  # 0-based indices

    selected_category = categories[category_index_choice]
    selected_question = \
        questions[questions.category == selected_category] \
        .iloc[question_index_choice]
    print("{} for {}: {}" \
        .format(selected_category, selected_question.value, \
            selected_question.question), end="\n>>> Who/What/Where/When is ")
    your_answer = input()
    if your_answer == selected_question.answer:
        print("Correct!")
        wallet += selected_question.value_float
    elif your_answer.lower() == "q":
        return False
    else:
        print("Incorrect. The answer is {}".format(selected_question.answer))
    return True
while play_one_question():
    answers_given += 1
print("You walk away with ${}".format(wallet))


Your wallet: $0
Select category (1-7, from left to right, or Q to quit)
>>> Select question (1-5, from top to bottom, or Q to quit)
>>> THAT'S THE SPOT for $200: This stickier version of "Jurassic Park" is at 5801 Wilshire Boulevard in Los Angeles
>>> Who/What/Where/When is Incorrect. The answer is La Brea Tar Pits
Your wallet: $0
Select category (1-7, from left to right, or Q to quit)
>>> Invalid choice. Please enter a number.
Select question (1-5, from top to bottom, or Q to quit)
>>> CHILDREN'S BOOK TITLES for $400: Margery Williams' story of a stuffed toy that comes to life: "The ____ Rabbit"
>>> Who/What/Where/When is Incorrect. The answer is Velveteen
Your wallet: $0
Select category (1-7, from left to right, or Q to quit)
>>> Select question (1-5, from top to bottom, or Q to quit)
>>> PROTESTANTISM for $200: Many of the Protestant denominations got their start during this 16th Century movement
>>> Who/What/Where/When is You walk away with $0
